In [ ]:
import re
import pandas as pd

In [ ]:

def extract_attribute(text):
    # Split the text into lines
    lines = text.split('\n')

    # Initialize variable to store product name
    product_name = "Not found"
    company_name = 'Not found'
    chemical_name = 'Not found'
    substance_use = 'Not found'
    cas = []
    ec = []
    hazard = ''
    hazard_lines = []
    in_hazard_section = False
    un = 'Not found'

    # Search for the line containing "Product name"
    for line in lines:
        if "Product name" in line or 'Trade name' in line:
            # Extract the product name from the line
            product_name = line.split(":")[1].strip()
        if "Company name of supplier" in line or 'Company ' in line:
            company_name = line.split(":")[1].strip()
        if 'Supplier :' in line:
            index = lines.index(line)+1
            company_name = lines[index].strip()
        if 'Substance name' in line or 'Chemical name' in line or 'Chemical Name' in line:
          if ':' in line:
            chemical_name = line.split(":", 1)[1].strip()
        if "Use of the" in line or 'Recommended use' in line or 'Specific end use(s)' in line:
            # Extract the use of the substance/mixture from the next line
            if ":" in line:
                substance_use = line.split(":", 1)[1].strip()
            else:
                index = lines.index(line) + 1
                if index < len(lines):
                    line = lines[index].strip()
                    if ":" in line:
                        substance_use = line.split(":", 1)[1].strip()
                    else:
                      next_index = index+1
                      if next_index<len(lines):
                        next_line = lines[next_index].strip()
                        substance_use = next_line.split(":", 1)[1].strip()
        if "SECTION 2" in line:
            in_hazard_section = True
        else:
          if "SECTION 3" in line:
            in_hazard_section = False
          else:
            if in_hazard_section == True:
              if 'Category' in line:
                hazard = 1
                break
              else:
                hazard = 0

        if 'UN number' in line:
          if ':' in line:
            un = line.split(":", 1)[1].strip()
          else:
            index = lines.index(line) +1
            if index< len(lines):
              un = lines[index].strip()


    cas = re.findall(r'\d+ -\d{2}-\d',text) or re.findall(r'\d+-\d{2}-\d', text)
    ec = re.findall(r'\d+ -\d{3}-\d\b',text) or re.findall(r'\d+-\d{3}-\d\b',text)


    return product_name, company_name, chemical_name, substance_use, cas, ec, hazard, un

In [ ]:
columns = ['Product name','Company name', 'Chemical', 'Market segmentation', 'CAS-No', 'EC-No', 'Hazard', 'UN number']
df= pd.DataFrame(columns=columns)

In [ ]:
columns = ['Product name','Company name', 'Chemical', 'Market segmentation', 'CAS-No', 'EC-No', 'Hazard', 'UN number']
df= pd.DataFrame(columns=columns)
def text_to_df(text):
  global df
  product_name, company_name, chemical_name, substance_use, cas, ec, hazard, un = extract_attribute(text)
  data = {
            'Product name': [product_name],
            'Company name': [company_name],
            'Chemical':[chemical_name],
            'Market segmentation': [substance_use],
            'CAS-No': [", ".join(cas)],
            'EC-No': [", ".join(ec)],
            'Hazard': [hazard],
            'UN number': [un]
    }
  df = pd.concat([df, pd.DataFrame(data)], ignore_index=True)
  return df



In [ ]:
columns = ['Product name','Company name', 'Chemical', 'Market segmentation', 'CAS-No', 'EC-No', 'Hazard', 'UN number']
df= pd.DataFrame(columns=columns)


In [ ]:
!pip install pypdf

In [ ]:
from pypdf import PdfReader
pdf_files = ['/content/Roquette_SDS_TW_复配增稠稳定水分保持剂 - LYCAGEL VS 720 PMX_000000202290_EN.PDF', '/content/Roquette_SDS_NL_CLEARAM® CI 20 00_000000200211_EN.PDF']
text_list=[]
for pdf in pdf_files:
  reader = PdfReader(pdf)
  text = ""
  for page in reader.pages:
      text += page.extract_text()

  text_list.append(text)

In [ ]:
for text in text_list:
  text_to_df(text)

df

,Product name,Company name,Chemical,Market segmentation,CAS-No,EC-No,Hazard,UN number
0,复配增稠 稳定水分保持 剂 - LYCAGEL VS 720 PMX Product sh...,"Roquette Biotech Nutritionals (Wuhan) Co., Ltd.",Not found,Pharmaceuticals. Nutraceuticals,"9049 -76-7, 9000 -07-1, 7558 -79-4",,0,Not found
1,CLEARAM ® CI 20 00 Product No.,ROQUETTE FRERES,Not found,"Human nutrition, Industrial. , Animal Feed. ,",55963 -33-2,,0,Not applicable .
